# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
# Helium at the center of the box
atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model = model_LDA(lattice, atoms, positions; symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
res   = self_consistent_field(basis; tol)
μref  = dipole(basis, res.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770233714388                   -0.53    9.0         
  2   -2.771677695329       -2.84       -1.30    1.0    111ms
  3   -2.771713774679       -4.44       -2.68    1.0    121ms
  4   -2.771714708629       -6.03       -3.66    2.0    128ms
  5   -2.771714714473       -8.23       -4.09    2.0    106ms
  6   -2.771714715224       -9.12       -4.95    1.0    110ms
  7   -2.771714715249      -10.59       -5.91    2.0    109ms
  8   -2.771714715250      -12.81       -6.25    2.0    124ms
  9   -2.771714715250      -13.72       -6.98    1.0    107ms
 10   -2.771714715250   +  -14.21       -7.56    1.0    114ms
 11   -2.771714715250      -13.85       -8.71    2.0    116ms


-0.00013457359078061386

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_LDA(lattice, atoms, positions;
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770489740978                   -0.52    9.0         
  2   -2.771778570302       -2.89       -1.32    1.0    101ms
  3   -2.771801677310       -4.64       -2.46    1.0    139ms
  4   -2.771801998066       -6.49       -3.17    1.0   90.9ms
  5   -2.771802073623       -7.12       -3.97    2.0    106ms
  6   -2.771802074174       -9.26       -4.23    1.0    118ms
  7   -2.771802074464       -9.54       -5.09    1.0   95.7ms
  8   -2.771802074474      -11.00       -5.34    2.0    128ms
  9   -2.771802074476      -11.78       -6.26    1.0    101ms
 10   -2.771802074476      -13.43       -6.54    1.0    117ms
 11   -2.771802074476      -14.75       -7.21    1.0    112ms
 12   -2.771802074476      -13.74       -7.72    2.0    124ms
 13   -2.771802074476   +  -13.94       -7.89    1.0    114ms
 14   -2.771802074476      -14.45       -8.99    1.0    109ms


0.017612221490443992

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457359078061386
Displaced dipole:  0.017612221490443992
Polarizability :   1.7746795081224607


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response
Now we use linear response to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field). Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

In [5]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; res.ρ)
    χ0δV = apply_χ0(res, δV)
    δρ - χ0δV
end

# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(res, δVext)

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve in iter 1; step 1: normres = 2.493920641259e-01
[ Info: GMRES linsolve in iter 1; step 2: normres = 3.766551116091e-03
[ Info: GMRES linsolve in iter 1; step 3: normres = 2.852764709669e-04
[ Info: GMRES linsolve in iter 1; step 4: normres = 4.694593836267e-06
[ Info: GMRES linsolve in iter 1; step 5: normres = 1.088787403407e-08
[ Info: GMRES linsolve in iter 1; step 6: normres = 6.273304126544e-11
[ Info: GMRES linsolve in iter 1; step 7: normres = 6.442697434628e-13
[ Info: GMRES linsolve in iter 1; finished at step 7: normres = 6.442697434628e-13
[ Info: GMRES linsolve in iter 2; step 1: normres = 1.091674287926e-09
[ Info: GMRES linsolve in iter 2; step 2: normres = 3.162459972864e-11
[ Info: GMRES linsolve in iter 2; step 3: normres = 4.672293423282e-12
[ Info: GMRES linsolve in iter 2; step 4: normres = 5.947737247171e-14
[ Info: GMRES linsolve in iter 2; finished at step 4: normres = 5.947737247171e-14
┌ Info: GMRES linsolve converged at iteration 2, step

As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.